<a href="https://colab.research.google.com/github/GabzBarbosa/Analise-de-Sentimento-para-review-de-produtos/blob/main/Analise_de_Sentimento_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip -q install google-genai

In [6]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [7]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [8]:
import pandas as pd
import numpy as np # Importado para lidar com NaN
from IPython.display import HTML, Markdown
import google.generativeai as genai
import json
import os

# --- Configuração da API do Gemini ---
# É altamente recomendável carregar sua chave de forma segura, por exemplo, de variáveis de ambiente.
# genai.configure(api_key="SUA_CHAVE_API_DO_GEMINI")
# ou, se você tiver configurado a variável de ambiente GOOGLE_API_KEY
genai.configure()

# --- Carregar o modelo Gemini ---
# O modelo 'gemini-1.5-pro' é geralmente mais robusto para gerar JSON estruturado.
# Se você tiver acesso ao 'gemini-2.0-flash' e ele funcionar para saída JSON, pode usá-lo.
MODEL_ID = "gemini-2.0-flash"
model = genai.GenerativeModel(MODEL_ID)

# --- Instalação de biblioteca (executar apenas uma vez) ---
# Garante que openpyxl esteja instalado para exportar para .xlsx
try:
    import openpyxl
except ImportError:
    print("Instalando openpyxl...")
    !pip install -q openpyxl
    import openpyxl
    print("openpyxl instalado com sucesso.")



# --- Função para carregar a planilha ---
def carregar_planilha(caminho_arquivo):
    """
    Carrega uma planilha (CSV ou Excel) e retorna um DataFrame do pandas.
    """
    try:
        if caminho_arquivo.endswith('.csv'):
            df = pd.read_csv(caminho_arquivo)
        elif caminho_arquivo.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(caminho_arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado. Use .csv, .xls ou .xlsx")
        return df
    except FileNotFoundError:
        print(f"Erro: Arquivo '{caminho_arquivo}' não encontrado.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao carregar a planilha: {e}")
        return None

# --- Função para salvar o relatório em XLSX ---
def salvar_relatorio_xlsx(nome_arquivo, df_relatorio):
    """
    Salva o DataFrame fornecido em um arquivo Excel (.xlsx).
    """
    try:
        df_relatorio.to_excel(nome_arquivo, index=False, engine='openpyxl')
        print(f"Relatório salvo com sucesso em '{nome_arquivo}'")
    except Exception as e:
        print(f"Ocorreu um erro ao salvar o relatório em XLSX: {e}")

# --- Caminho para sua planilha de entrada ---
caminho_da_planilha = '/content/Teste Analise de Sentimento - Página1 (8).csv'
# --- Nome do arquivo para salvar o relatório de saída (XLSX) ---
nome_do_relatorio_xlsx = 'relatorio_analise_qualidade_clientes.xlsx'

# Carrega os dados da planilha
df_avaliacoes = carregar_planilha(caminho_da_planilha)

if df_avaliacoes is not None:
    # Garante que as colunas 'id', 'nota', 'comentario' e 'Recomenda produto' existam
    colunas_necessarias = ['id', 'nota', 'comentario', 'Recomenda produto']
    if not all(coluna in df_avaliacoes.columns for coluna in colunas_necessarias):
        print(f"Erro: Uma ou mais colunas necessárias ({', '.join(colunas_necessarias)}) não foram encontradas na planilha.")
    else:
        # CONVERSÃO DA COLUNA 'Recomenda produto' PARA NUMÉRICO (1 para 'Sim', 0 para 'Não')
        # Lida com valores nulos (NaN) preenchendo-os com 0 antes da conversão para int
        df_avaliacoes['Recomenda produto'] = df_avaliacoes['Recomenda produto'].astype(str).str.lower().map({'sim': 1, 'não': 0}).fillna(0).astype(int)

        # Pré-processamento dos dados para consolidar por ID
        dados_consolidados_para_gemini = []
        for id_cliente, grupo in df_avaliacoes.groupby('id'):
            media_nota = grupo['nota'].mean()
            # Garante que o percentual seja calculado apenas se houver avaliações no grupo
            percentual_recomendacao = (grupo['Recomenda produto'].sum() / len(grupo) * 100) if len(grupo) > 0 else 0

            # Converte os comentários para uma lista, substituindo NaN por None
            comentarios_cliente = grupo['comentario'].replace({np.nan: None}).tolist()

            dados_consolidados_para_gemini.append({
                "id_cliente": str(id_cliente), # Garante que o ID seja uma string no JSON
                "media_nota": round(media_nota, 2),
                "percentual_recomendacao": round(percentual_recomendacao, 2),
                "comentarios": comentarios_cliente
            })

        # Converte a lista de dicionários para uma string JSON formatada
        json_input_para_gemini = json.dumps(dados_consolidados_para_gemini, indent=2, ensure_ascii=False)

        if json_input_para_gemini.strip():
            # --- Chamar a API do Gemini para a curadoria e resumo em JSON ---
            prompt_curadoria = f"""
            Você é um analista de qualidade. Abaixo estão avaliações de clientes em formato JSON, consolidadas por ID, incluindo média de nota, percentual de recomendação e os comentários brutos.

            Sua tarefa é:
            1. Para cada cliente (ID), gerar um resumo conciso do feedback e identificar as principais sugestões de melhoria específicas para o produto/serviço relacionado a esse cliente.
            2. O resultado DEVE ser um objeto JSON. Este objeto JSON deve conter uma lista de objetos, onde cada objeto representa um cliente analisado.
            3. Cada objeto de cliente no JSON DEVE ter as seguintes chaves (sempre presentes):
               - "id_cliente": (string) O ID do cliente.
               - "Quantidade_Reviews": (Numeric) Conta o total de avaliações relacionadas a id_cliente.
               - "media_nota": (Numeric) Média total das notas relacionadas a id_cliente.
               - "resumo_feedback": (string) Um resumo conciso do feedback do cliente.
               - "sugestoes_melhoria": (array de strings) Uma lista de sugestões de melhoria específicas para o produto/serviço relacionadas ao feedback deste cliente. Se não houver sugestões, use um array vazio `[]`.
               - "sentimento_geral": (string) O sentimento geral da avaliação (ex: "Positivo", "Neutro", "Negativo").

            Exemplo de formato de saída JSON:
            ```json
            [
              {{
                "id_cliente": "ClienteA",
                "Quantidade_Reviews": "23 Avaliações Recebidas"
                "media_nota": "1.5",
                "resumo_feedback": "Cliente satisfeito com a entrega rápida, mas achou o preço alto.",
                "sugestoes_melhoria": ["Revisar precificação", "Destacar custo-benefício"],
                "sentimento_geral": "Neutro"
              }},
              {{
                "id_cliente": "ClienteB",
                "Quantidade_Reviews": "2 Avaliações Recebidas"
                "media_nota": "4.5",
                "resumo_feedback": "Produto de excelente qualidade e ótimo atendimento. Recomenda fortemente.",
                "sugestoes_melhoria": [],
                "sentimento_geral": "Positivo"
              }}
            ]
            ```

            Dados dos Clientes (JSON):
            {json_input_para_gemini}
            """

            print("Gerando relatório de curadoria de comentários com Gemini (esperando JSON)...")
            try:
                resposta_gemini = model.generate_content(
                    prompt_curadoria,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0.2,
                        max_output_tokens=4000,
                        response_mime_type="application/json"
                    )
                )

                json_string_resposta = resposta_gemini.text

                print("\n--- Resposta JSON do Gemini (para depuração) ---\n")
                print(json_string_resposta)
                print("\n-------------------------------------------------\n")

                dados_relatorio = json.loads(json_string_resposta)

                df_relatorio_final = pd.DataFrame(dados_relatorio)

                display(Markdown(f"## Relatório de Análise de Qualidade (Dados Tabulares):\n"))
                display(df_relatorio_final)

                # --- Salvar o relatório gerado em XLSX ---
                salvar_relatorio_xlsx(nome_do_relatorio_xlsx, df_relatorio_final)

            except json.JSONDecodeError as e:
                print(f"Erro: A resposta do Gemini não é um JSON válido. Verifique o prompt e a resposta. Erro: {e}")
                print(f"Resposta bruta recebida: {resposta_gemini.text}")
            except Exception as e:
                print(f"Ocorreu um erro ao chamar a API do Gemini ou processar o relatório: {e}")
        else:
            print("Não há dados válidos para processar e gerar o relatório.")
else:
    print("Não foi possível carregar a planilha. Verifique o caminho e o formato do arquivo.")

Gerando relatório de curadoria de comentários com Gemini (esperando JSON)...

--- Resposta JSON do Gemini (para depuração) ---

[
  {
    "id_cliente": "FB9-9335",
    "Quantidade_Reviews": 200,
    "media_nota": 4.59,
    "resumo_feedback": "Os clientes estão geralmente satisfeitos com a qualidade, aparência e custo-benefício do tênis. Muitos elogiam a entrega rápida e o design do produto. No entanto, alguns clientes relatam desconforto inicial, problemas com o tamanho (forma pequena ou grande), e preocupações sobre a autenticidade do produto. Alguns mencionam que o tênis é duro, amassa facilmente ou machuca o pé.",
    "sugestoes_melhoria": [
      "Melhorar o conforto do tênis, especialmente na sola e nas laterais.",
      "Revisar a tabela de tamanhos para garantir maior precisão.",
      "Garantir a autenticidade dos produtos e informar claramente se não forem originais.",
      "Melhorar a qualidade dos materiais para evitar amassados e desgaste rápido.",
      "Prestar mais aten

## Relatório de Análise de Qualidade (Dados Tabulares):


,id_cliente,Quantidade_Reviews,media_nota,resumo_feedback,sugestoes_melhoria,sentimento_geral
0,FB9-9335,200,4.59,Os clientes estão geralmente satisfeitos com a...,"[Melhorar o conforto do tênis, especialmente n...",Positivo
1,NQQ-4378,100,4.68,"Os clientes apreciam a aparência, qualidade e ...",[Melhorar a precisão das informações sobre o t...,Positivo
2,NQQ-4379,100,4.56,Os clientes geralmente consideram o tênis esti...,"[Melhorar o conforto do tênis, especialmente n...",Positivo


Relatório salvo com sucesso em 'relatorio_analise_qualidade_clientes.xlsx'
